# Introduction

This notebook demonstrates how to predict likely n- and p-type dopant atoms using pymatgen. This example uses the Materials API to download the structure of interest but any `Structure` object can be used. Two methods for choosing dopants are demonstrated. The first uses a simple Shannon radii comparison, whereas the second is based on the substitution probability of two atoms calculated using the `SubstitutionPredictor` utility in pymatgen. This code requires knowledge of the oxidation state of all elements in the structure. These can be guessed using pymatgen but should be checked to ensure the validity of the results.

*Author: Alex Ganose (10/06/18)*

In [ ]:
# Uncomment the subsequent lines in this cell to install dependencies for Google Colab.
# !pip install pymatgen==2022.7.19

In [ ]:
# Imports we need for generating dopant suggestions
from __future__ import annotations

from pprint import pprint

from pymatgen.analysis.local_env import CrystalNN
from pymatgen.analysis.structure_prediction.dopant_predictor import (
    get_dopants_from_shannon_radii,
    get_dopants_from_substitution_probabilities,
)
from pymatgen.ext.matproj import MPRester

In [ ]:
# Establish rester for accessing Materials API

api_key = None  # INSERT YOUR OWN API KEY

mpr = MPRester()

Here we define a variable -- `num_dopants` for how many dopants you wish to explore.

In [ ]:
num_dopants = 5  # number of highest probability dopants you wish to see

## Download a structure and add oxidation states

In this section, we use the Materials API to download a structure and add information on the oxidation states of the atoms.

In [ ]:
mp_id = "mp-856"  # Materials Project id for rutile SnO2

structure = mpr.get_structure_by_material_id(mp_id)

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

The downloaded structure does not contain oxidation state information. There are two ways to add this information. The first is to specify the oxidation state of the elements manually.

In [ ]:
structure.add_oxidation_state_by_element({"Sn": 4, "O": -2})

Structure Summary
Lattice
    abc : 3.20749977 4.76383605 4.76580004
 angles : 90.0 90.0 90.0
 volume : 72.82143907393863
      A : 3.20749977 0.0 0.0
      B : 0.0 4.76383605 -0.0
      C : 0.0 -0.0 4.76580004
    pbc : True True True
PeriodicSite: Sn (Sn4+) (1.604, 2.382, 2.383) [0.5, 0.5, 0.5]
PeriodicSite: Sn (Sn4+) (0.0, 0.0, 0.0) [0.0, -0.0, 0.0]
PeriodicSite: O (O2-) (1.604, 0.9252, 3.842) [0.5, 0.1942, 0.8062]
PeriodicSite: O (O2-) (1.604, 3.839, 0.9238) [0.5, 0.8058, 0.1938]
PeriodicSite: O (O2-) (0.0, 1.457, 1.459) [-0.0, 0.3058, 0.3062]
PeriodicSite: O (O2-) (0.0, 3.307, 3.307) [-0.0, 0.6942, 0.6938]

Alternatively, we can use pymatgen to guess the oxidation states. If using this method you should check that the oxidation states are what you expect.

In [ ]:
structure.add_oxidation_state_by_guess()

Structure Summary
Lattice
    abc : 3.20749977 4.76383605 4.76580004
 angles : 90.0 90.0 90.0
 volume : 72.82143907393863
      A : 3.20749977 0.0 0.0
      B : 0.0 4.76383605 -0.0
      C : 0.0 -0.0 4.76580004
    pbc : True True True
PeriodicSite: Sn (Sn4+) (1.604, 2.382, 2.383) [0.5, 0.5, 0.5]
PeriodicSite: Sn (Sn4+) (0.0, 0.0, 0.0) [0.0, -0.0, 0.0]
PeriodicSite: O (O2-) (1.604, 0.9252, 3.842) [0.5, 0.1942, 0.8062]
PeriodicSite: O (O2-) (1.604, 3.839, 0.9238) [0.5, 0.8058, 0.1938]
PeriodicSite: O (O2-) (0.0, 1.457, 1.459) [-0.0, 0.3058, 0.3062]
PeriodicSite: O (O2-) (0.0, 3.307, 3.307) [-0.0, 0.6942, 0.6938]

Let's check what oxidation states pymatgen guessed.

In [ ]:
species = structure.composition.elements

print(species)

[Species Sn4+, Species O2-]


## Finding dopants by Shannon radii

In this section, we use the known Shannon radii to predict likely dopants. We will prefer dopants which have the smallest difference in radius to the host atoms. As the Shannon radii depend on the coordination number of the site, we must first calculate the bonding in the structure. In this example, we do this using the `CrystalNN` class.

In [ ]:
cnn = CrystalNN()
bonded_structure = cnn.get_bonded_structure(structure)

Pymatgen has a function to take a bonded structure with oxidation states and report the closest n- and p-type dopants, sorted by the difference in Shannon radii. Let's run this on our bonded structure:

In [ ]:
dopants = get_dopants_from_shannon_radii(bonded_structure, num_dopants=num_dopants)

pprint(dopants)

{'n_type': [{'dopant_species': Species U6+,
             'original_species': Species Sn4+,
             'radii_diff': 0.040000000000000036},
            {'dopant_species': Species Nb5+,
             'original_species': Species Sn4+,
             'radii_diff': -0.04999999999999993},
            {'dopant_species': Species Ta5+,
             'original_species': Species Sn4+,
             'radii_diff': -0.04999999999999993},
            {'dopant_species': Species F-,
             'original_species': Species O2-,
             'radii_diff': -0.06000000000000005},
            {'dopant_species': Species Np5+,
             'original_species': Species Sn4+,
             'radii_diff': 0.06000000000000005}],
 'p_type': [{'dopant_species': Species Ni2+,
             'original_species': Species Sn4+,
             'radii_diff': 0.0},
            {'dopant_species': Species Ru3+,
             'original_species': Species Sn4+,
             'radii_diff': -0.009999999999999898},
            {'dopant_speci

The most favoured n-type dopant is U on a Sn site. Unfortunately, this is not a sustainable or safe choice of dopant. The most common industrial n-type dopant for SnO2 is fluorine. While F is present in our list of suggested dopants, it found way down at suggestion number 4.

Another limitation of the Shannon radii approach to choosing dopants is that the radii depend on both the coordination number and charge state. For many elements, the radii for many charge state/coordination number combinations have not been tabulated, meaning this approach is incomplete.

Instead we should use a more robust approach to determine possible dopants. 

## Finding dopants by substitution probability

In this section, we use the statistics provided by `SubstitutionPredictor` to predict likely dopants substitutions using a data-mined approach from ICSD data. Based on the species in the structure, we get a list of which species are likely to substitute in but have different charge states. The substitution prediction methodology is presented in: 
*Hautier, G., Fischer, C., Ehrlacher, V., Jain, A., and Ceder, G. (2011) Data Mined Ionic Substitutions for the Discovery of New Compounds. Inorganic Chemistry, 50(2), 656-663. doi:10.1021/ic102031h*

Here, we define a variable -- `threshold` for the threshold probability in making substitution/structure predictions.

In [ ]:
threshold = 0.001  # probability threshold for substitution/structure predictions

Pymatgen provides a function to filter the predicted substitutions by their charge states and return a list of n- and p-type dopants. Let's run the function on the structure we downloaded earlier:

In [ ]:
dopants = get_dopants_from_substitution_probabilities(
    structure, num_dopants=num_dopants, threshold=threshold
)

pprint(dopants)

{'n_type': [{'dopant_species': Species F-,
             'original_species': Species O2-,
             'probability': 0.06692682583342514},
            {'dopant_species': Species Cl-,
             'original_species': Species O2-,
             'probability': 0.02102263828843219},
            {'dopant_species': Species Ta5+,
             'original_species': Species Sn4+,
             'probability': 0.019486221245908535},
            {'dopant_species': Species Sb5+,
             'original_species': Species Sn4+,
             'probability': 0.010380692735493776},
            {'dopant_species': Species Nb5+,
             'original_species': Species Sn4+,
             'probability': 0.009988531781437172}],
 'p_type': [{'dopant_species': Species Co2+,
             'original_species': Species Sn4+,
             'probability': 0.023398867249112977},
            {'dopant_species': Species Cd2+,
             'original_species': Species Sn4+,
             'probability': 0.022644061067779386},
     

The function returns a list of potential dopants sorted by their substitution probability. The most likely n-type dopant is F on a O site. Fluorine doped SnO2 (FTO) is one of the most widely used transparent conducting oxides, therefore validating this approach.